In [174]:
import soundfile
import pydub
import librosa
import numpy as np
import io
import os
import uuid
from pydub.playback import play

In [175]:
src = 'filter-test-dataset/Backyard-2024-Feb-12-16/0101/050711.wav'
a = soundfile.read(src)
b = librosa.load(src, sr = None)
assert len(a) == 2 and len(a) == len(b)
assert (a[0] == b[0]).all()
assert a[1] == b[1]

In [176]:
def wav_to_mp3_to_wav(wav: np.ndarray, sr: int) -> np.ndarray:
    f = 'wmw-cvt-{uuid.uuid4()}'
    
    try:
        # doing the exact same thing in-memory gives garbled audio... whatever...
        soundfile.write(f, wav, sr, format = 'WAV')
        seg = pydub.AudioSegment.from_wav(f)
        seg.export(f, format = 'mp3')
        seg = pydub.AudioSegment.from_mp3(f)
        seg.export(f, format = 'wav')
        res = soundfile.read(f)
        res = (librosa.resample(res[0], orig_sr = res[1], target_sr = sr), sr)
        
        assert res[0].dtype == wav.dtype, f'{wav.dtype} -> {res[0].dtype}'
        assert res[0].shape == wav.shape, f'{wav.shape} -> {res[0].shape}'
        assert res[1] == sr, f'{sr} -> {res[1]}'
        return res[0]
    finally:
        os.remove(f)

x = wav_to_mp3_to_wav(*a)
print(a[0])
print(x)

[ 0.00158691  0.00640869  0.00585938 ... -0.00079346 -0.00061035
 -0.00048828]
[ 0.00255525  0.00497464  0.00557264 ... -0.00070736 -0.00051302
 -0.00024977]


In [177]:
play(pydub.AudioSegment(a[0].astype('float32').tobytes(), frame_rate = a[1], sample_width = 4, channels = 1))

ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave


In [178]:
play(pydub.AudioSegment(x.astype('float32').tobytes(), frame_rate = a[1], sample_width = 4, channels = 1))

ALSA lib pcm_dsnoop.c:641:(snd_pcm_dsnoop_open) unable to open slave
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.rear
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.center_lfe
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM cards.pcm.side
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_oss.c:377:(_snd_pcm_oss_open) Unknown field port
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_usb_stream.c:486:(_snd_pcm_usb_stream_open) Invalid type for card
ALSA lib pcm_dmix.c:1089:(snd_pcm_dmix_open) unable to open slave
